# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
# import numpy and pandas
import pandas as pd
import numpy as np

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents

In [2]:
# Run this code:
salaries = pd.read_csv('../Current_Employee_Names__Salaries__and_Position_Titles.csv')
salaries.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [3]:
# Your code here:
salaries.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33183 entries, 0 to 33182
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               33183 non-null  object 
 1   Job Titles         33183 non-null  object 
 2   Department         33183 non-null  object 
 3   Full or Part-Time  33183 non-null  object 
 4   Salary or Hourly   33183 non-null  object 
 5   Typical Hours      8022 non-null   float64
 6   Annual Salary      25161 non-null  float64
 7   Hourly Rate        8022 non-null   float64
dtypes: float64(3), object(5)
memory usage: 2.0+ MB


In [4]:
# Tenemos muchísimos valores nulos en las columnas Typical Hours y Hourly Rate y una cantidad razonablemente grande también en la columna Annual Salary

Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [5]:
# Your code here:
salaries['Full or Part-Time'].value_counts()


Full or Part-Time
F    31090
P     2093
Name: count, dtype: int64

What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [7]:
salaries.Department.unique()

array(['POLICE', 'GENERAL SERVICES', 'WATER MGMNT', 'OEMC',
       'CITY COUNCIL', 'AVIATION', 'STREETS & SAN', 'FIRE',
       'FAMILY & SUPPORT', 'PUBLIC LIBRARY', 'TRANSPORTN',
       "MAYOR'S OFFICE", 'HEALTH', 'BUSINESS AFFAIRS', 'LAW', 'FINANCE',
       'CULTURAL AFFAIRS', 'COMMUNITY DEVELOPMENT', 'PROCUREMENT',
       'BUILDINGS', 'ANIMAL CONTRL', 'CITY CLERK', 'BOARD OF ELECTION',
       'DISABILITIES', 'HUMAN RESOURCES', 'DoIT', 'BUDGET & MGMT',
       'TREASURER', 'INSPECTOR GEN', 'HUMAN RELATIONS', 'COPA',
       'BOARD OF ETHICS', 'POLICE BOARD', 'ADMIN HEARNG',
       'LICENSE APPL COMM'], dtype=object)

In [6]:
# Your code here:
salaries.Department.value_counts()


Department
POLICE                   13414
FIRE                      4641
STREETS & SAN             2198
OEMC                      2102
WATER MGMNT               1879
AVIATION                  1629
TRANSPORTN                1140
PUBLIC LIBRARY            1015
GENERAL SERVICES           980
FAMILY & SUPPORT           615
FINANCE                    560
HEALTH                     488
CITY COUNCIL               411
LAW                        407
BUILDINGS                  269
COMMUNITY DEVELOPMENT      207
BUSINESS AFFAIRS           171
COPA                       116
BOARD OF ELECTION          107
DoIT                        99
PROCUREMENT                 92
INSPECTOR GEN               87
MAYOR'S OFFICE              85
CITY CLERK                  84
ANIMAL CONTRL               81
HUMAN RESOURCES             79
CULTURAL AFFAIRS            65
BUDGET & MGMT               46
ADMIN HEARNG                39
DISABILITIES                28
TREASURER                   22
HUMAN RELATIONS             

# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [27]:
# Por lo tanto la H0 es que el hourly wage es $30/hr. Vamos a utilizar el estadístico Z test.
import pylab as plt
from scipy.stats import norm
from statsmodels.stats import weightstats as w


In [28]:
salaries.describe().T

,count,mean,std,min,25%,50%,75%,max
Typical Hours,8022.0,34.507604,9.252077,10.00,20.0,40.0,40.0,40.0
Annual Salary,25161.0,86786.999790,21041.354602,7200.00,76266.0,90024.0,96060.0,300000.0
Hourly Rate,8022.0,32.788558,12.112573,2.65,21.2,35.6,40.2,109.0


In [31]:
#help(w)

In [32]:
# Nos piden un hypothesis test al 95% y con un two sided interval -> O mu = 30 | mu != 30

# Sabemos que la bbdd tiene 33183 entradas -> 33183 asalariados -> n=33183 
empleados = 33183
mu = 30

In [36]:
# Your code here:
z_test = w.ztest(salaries['Hourly Rate'].dropna(), value=mu, alternative='two-sided')
z_test

(20.6198057854942, 1.8228873859286195e-94)

In [64]:
# Voy a probar a hacerlo con el t test
t_test = ttest_1samp(salaries['Hourly Rate'].dropna(), 30, alternative='two-sided')
t_test

TtestResult(statistic=20.6198057854942, pvalue=4.3230240486229894e-92, df=8021)

In [54]:
#help(w.ztest)

'''
Returns
tstat : float
    test statistic
pvalue : float
    pvalue of the t-test


'two-sided': H1: difference in means not equal to value (default)
'larger' :   H1: difference in means larger than value
'smaller' :  H1: difference in means smaller than value
'''


"\nReturns\ntstat : float\n    test statistic\npvalue : float\n    pvalue of the t-test\n\n\n'two-sided': H1: difference in means not equal to value (default)\n'larger' :   H1: difference in means larger than value\n'smaller' :  H1: difference in means smaller than value\n"

In [40]:
# El ztest nos devuelve en primer lugar el estadístico y en 2º el pvalue. 

# Al ser el pvalue<=0,5 podemos rechazar la hipótesis nula y afirmar que el hourly wage es diferente de $30/hr

# La manera correcta de decirlo sería: hay evidencia suficiente para sugerir que la diferencia observada es estadísticamente significativa :)

In [ ]:
'''
En resumen, el Z-test es más apropiado cuando la desviación estándar poblacional es conocida, mientras que el t-test 
se utiliza cuando la desviación estándar poblacional no es conocida y, especialmente, cuando el tamaño de la muestra 
es pequeño. El t-test es más robusto en situaciones donde la variabilidad de la muestra debe ser tenida en cuenta 
debido al tamaño limitado de la muestra.

'''

We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [48]:
# En este caso la hipótesis nula es que el salario medio anual es mayor al del año pasado -> mu > $86.000 -> Hipótesis alternativa es que ese salario anual es menor

# Esta vez nos piden un intervalo de confianza del 95%, one sided.

# Vamos a crear una columna con unicamente los salarios de los polis:

polis = salaries['Annual Salary'][salaries.Department == 'POLICE']
polis.head()

0     101442.0
1      94122.0
9      93354.0
11     84054.0
12     87006.0
Name: Annual Salary, dtype: float64

In [49]:
mu = 86000

In [51]:
z_test = w.ztest(polis.dropna(), value=mu, alternative='smaller')
z_test

(3.081997005712994, 0.9989719154712452)

In [52]:
# Recordamos que el primer valor es el estadístico y el segundo el pvalor -> Pvalue=0.9989719154712452

# Al ser pvalor >= 0,05 no tenemos evidencia suficiente como para afirmar que la Hipótesis nula es incorrecta -> Creemos al poli

In [ ]:
# Un momento, he hecho este apartado con Z test por ser la muestra muy grande. Según la teoría, el estadístico z test se utiliza cuando conoces la desviación estandar poblacional
# o cuando la muestra es muy grande. En este caso no conocíamos la desv. est. poblacional (sí la muestral) pero es válido el test por el gran tamaño de la muestra.
# Vamos a hacer el mismo cálculo pero con el estadístico t:

In [56]:
from scipy.stats import ttest_1samp

In [58]:
#help(ttest_1samp)

In [62]:
t_test = ttest_1samp(polis.dropna(), mu, alternative='less')
t_test

TtestResult(statistic=3.081997005712994, pvalue=0.9989698298224517, df=13403)

In [63]:
# Obtenemos un pvalor=0.99896 (muy parecido por cierto al del z test), pvalor>0.05 por lo que no podemos rechazar la Hipótesis nula -> poli tenía razón

# mismo resultado :)

Using the `crosstab` function, find the department that has the most hourly workers. 

In [9]:
# Your code here:


The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [10]:
# Your code here:


# Challenge 3 - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [11]:
# Your code here:


Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [12]:
# Your code here:


# Challenge 4 - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [13]:
# Your code here:
